## Data Preprocessing
Separate training and test data into folders - 1 for each

Also then separate images into folders for each class, e.g. cats and dogs

Naming convention - LABEL.NUMBER.EXTENSION, e.g. cat.1.jpg

10000 images total - 80/20 split between training and test set

Unlike previous ANN, the data preprocessing is done manually and some feature selection is one later

## Building the CNN

In [1]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Conv2D #2D for images
from keras.layers import MaxPooling2D #As above
from keras.layers import Flatten
from keras.layers import Dense


Using TensorFlow backend.


In [2]:
# Initialising the CNN
classifier = Sequential()

In [3]:
# Step 1 - Convolution
#Create number of feature maps
#Convolution2D 
#- number of filters = 32 (common practise to start with a 32 and then add more conv layers with 64 filters for example)
#- number of rows in each filter = 3
#- number of columns in each filter
#- input shape - shape of the input image. Not all have the same size or format so we must convert all input images to same format
# Colour images - convert to 3D array. BW - 2D array
#(64, 64, 3) = 3 channels (colour, 1 for BW), 64x64 format (smaller as we are running on a CPU)
#- activation function - makes sure we have no negative pixel values in our feature maps, ensures non-linearity in model
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

In [4]:
# Step 2 - Pooling
#Reduces size of feature map
#Apply to each feature map as result of conv layer
#Reduce nodes needed for next step (flattening)
#Pool size 2x2 - reduces without losing too much feature information
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [5]:
# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

In [6]:
# Step 3 - Flattening
classifier.add(Flatten())

In [7]:
# Step 4 - Full connection
#Classic ANN of fully connected layers
#units = no. nodes in hidden layer.
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [8]:
# Compiling the CNN
#optimizer = SGD algorithm
#loss = loss function Binary Cross entropy used as our outcome is binary
#metrics = performance metric
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

## Fitting the CNN to the images

In [9]:
#Preprocessing
#Image Augmentation from keras
#Prevents overfitting
#ImageDataGenerator - generates batches of images with some random augmentations (rotation, flipped, etc.)
from keras.preprocessing.image import ImageDataGenerator

In [10]:
#Generate Augmented data
#rescale - feature scaling
#shear_range Shearing or transvection. Geometrical transformation
#zoom_range - random zooming
#horizontal_flip
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [11]:
#Create training and test data from augmented data generator
#target size - size expected by the model
#class mode - binary as we have only two classes
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [12]:
#Fit model to training set
#steps per epoch = number of training examples
classifier.fit_generator(training_set,
                         steps_per_epoch = 8000,
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 2000)

Epoch 1/25
8000/8000 [==============================] - 1342s - loss: 0.3704 - acc: 0.8255 - val_loss: 0.5233 - val_acc: 0.8152
Epoch 2/25
8000/8000 [==============================] - 1285s - loss: 0.1346 - acc: 0.9474 - val_loss: 0.8162 - val_acc: 0.8145
Epoch 3/25
8000/8000 [==============================] - 1286s - loss: 0.0755 - acc: 0.9723 - val_loss: 0.9337 - val_acc: 0.8123
Epoch 4/25
8000/8000 [==============================] - 1284s - loss: 0.0550 - acc: 0.9803 - val_loss: 0.9254 - val_acc: 0.8085
Epoch 5/25
8000/8000 [==============================] - 1286s - loss: 0.0453 - acc: 0.9841 - val_loss: 0.9858 - val_acc: 0.8090
Epoch 6/25
8000/8000 [==============================] - 1283s - loss: 0.0378 - acc: 0.9868 - val_loss: 1.0603 - val_acc: 0.8091
Epoch 7/25
8000/8000 [==============================] - 1283s - loss: 0.0327 - acc: 0.9886 - val_loss: 1.0846 - val_acc: 0.8194
Epoch 8/25
8000/8000 [==============================] - 1282s - loss: 0.0306 - acc: 0.9895 - val_loss: 1

## Homework - Make new predictions

In [13]:
import numpy as np
from keras.preprocessing import image

#target_size = 64, 64 to match our input dimensions
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))

#Converts image to 3D array 64x64x3
test_image = image.img_to_array(test_image)

#Add additional dimension - for batch
#1 image = 1 batch of one image
test_image = np.expand_dims(test_image, axis = 0)

#Make prediction
result = classifier.predict(test_image)

#Gives mapping between classes and indices in prediction
training_set.class_indices

#Converts 1/0 to class name
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [15]:
prediction

'dog'

In [16]:
#target_size = 64, 64 to match our input dimensions
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))

#Converts image to 3D array 64x64x3
test_image = image.img_to_array(test_image)

#Add additional dimension - for batch
#1 image = 1 batch of one image
test_image = np.expand_dims(test_image, axis = 0)

#Make prediction
result = classifier.predict(test_image)

#Converts 1/0 to class name
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'

In [17]:
prediction

'cat'

## Homework Challenge
Evaluation was already made during the training with the validation set, therefore k-Fold Cross Validation is not needed.

Then the techniques to improve and tune a CNN model are the same as for ANNs. So here is the challenge:

Apply the techniques you've learnt and use your architect power to make a CNN that will give you the gold medal:

Bronze medal: Test set accuracy between 80% and 85%

Silver medal: Test set accuracy between 85% and 90%

Gold medal: Test set accuracy over 90%!!

Rules:

- Use the same training set

- Dropout allowed

- Customized SGD allowed

- Specific seeds not allowed


In [2]:
#From Udemy - best answer - 92% accuracy
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
 
# Image dimensions
img_width, img_height = 150, 150 
 
"""
    Creates a CNN model
    p: Dropout rate
    input_shape: Shape of input 
"""
def create_model(p, input_shape=(32, 32, 3)):
    # Initialising the CNN
    model = Sequential()
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', input_shape=input_shape, activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(32, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    # Convolution + Pooling Layer 
    model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flattening
    model.add(Flatten())
    # Fully connection
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(p/2))
    model.add(Dense(1, activation='sigmoid'))
    
    # Compiling the CNN
    optimizer = Adam(lr=1e-3)
    metrics=['accuracy']
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=metrics)
    return model
"""
    Fitting the CNN to the images.
"""
def run_training(bs=32, epochs=10):
    
    train_datagen = ImageDataGenerator(rescale = 1./255, 
                                       shear_range = 0.2, 
                                       zoom_range = 0.2, 
                                       horizontal_flip = True)
    test_datagen = ImageDataGenerator(rescale = 1./255)
 
    training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (img_width, img_height),
                                                 batch_size = bs,
                                                 class_mode = 'binary')
                                                 
    test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (img_width, img_height),
                                            batch_size = bs,
                                            class_mode = 'binary')
                                            
    model = create_model(p=0.6, input_shape=(img_width, img_height, 3))                                  
    model.fit_generator(training_set,
                         steps_per_epoch=8000/bs,
                         epochs = epochs,
                         validation_data = test_set,
                         validation_steps = 2000/bs)

In [ ]:
run_training(bs=32, epochs=100)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/100
250/250 [==============================] - 277s - loss: 0.6942 - acc: 0.5015 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 2/100
250/250 [==============================] - 29252s - loss: 0.6935 - acc: 0.4940 - val_loss: 0.6931 - val_acc: 0.5100
Epoch 3/100
249/250 [============================>.] - ETA: 112s - loss: 0.6934 - acc: 0.4903